In [277]:
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before
import gzip
from collections import defaultdict
import pickle

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

allRatings = []
userRatings = defaultdict(list)
userItemRating = {}
itemUserRating = {}
count = 0
for l in readGz("train.json.gz"):
    if count >= 600000:
        break
    user,item = l['reviewerID'],l['itemID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])
    if item not in itemUserRating:
        itemUserRating[item] = {}
    itemUserRating[item][user] = l['rating']
    if user not in userItemRating:
        userItemRating[user] = {}
    userItemRating[user][item] = l['rating']
    count += 1
   
  
# pickle.dump(userItemRating, open( "userItemRating.p", "wb" ) )
# pickle.dump(itemUserRating, open( "itemUserRating.p", "wb" ) )
# pickle.dump(userRatings, open( "userRatings.p", "wb" ) )
# pickle.dump(allRatings, open( "allRatings.p", "wb" ) )

pickle.dump(userItemRating, open( "userItemRating_600k.p", "wb" ) )
pickle.dump(itemUserRating, open( "itemUserRating_600k.p", "wb" ) )


# globalAverage = sum(allRatings) / len(allRatings)
# userAverage = {}
# for u in userRatings:
#     userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

# predictions = open("predictions_Rating.txt", 'w')
# for l in open("pairs_Rating.txt"):
#   if l.startswith("userID"):
#     #header
#     predictions.write(l)
#     continue
#   u,i = l.strip().split('-')
#   if u in userAverage:
#     predictions.write(u + '-' + i + ',' + str(userAverage[u]) + '\n')
#   else:
#     predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')

# predictions.close()

In [2]:
import pickle
userItemRating = pickle.load( open( "userItemRating.p", "rb" ) )
itemUserRating = pickle.load( open( "itemUserRating.p", "rb" ) )
userItemRating_100k = pickle.load( open( "userItemRating_100k.p", "rb" ) )
itemUserRating_100k = pickle.load( open( "itemUserRating_100k.p", "rb" ) )
userItemRating_900k = pickle.load( open( "userItemRating_900k.p", "rb" ) )
itemUserRating_900k = pickle.load( open( "itemUserRating_900k.p", "rb" ) )
userItemRating_600k = pickle.load( open( "userItemRating_600k.p", "rb" ) )
itemUserRating_600k = pickle.load( open( "itemUserRating_600k.p", "rb" ) )

# print userItemRating

In [3]:
import math
import numpy as np
import gzip
from collections import defaultdict
import pickle
import random

#load pickles 
# userItemRating = pickle.load( open( "userItemRating.p", "rb" ) )
# itemUserRating = pickle.load( open( "itemUserRating.p", "rb" ) )
# subsetSize = 1000
# subsetCount = 0

# userItemRatingSubset = {}
# for user in userItemRating:
#     subsetCount += 1
#     if subsetCount == subsetSize:
#         break
#     for item in userItemRating[user]:
#         if user not in userItemRatingSubset:
#             userItemRatingSubset[user] = {}
#         userItemRatingSubset[user][item] = userItemRating[user][item]
        
        
# userItemRating = userItemRatingSubset
            
#objective
def f(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    obj = 0.0
    count = 0.0
    for user in userItemRating:
        for item in userItemRating[user]:
            if len(gamma_u[user]) != len(gamma_i[item]):
                print "len gamma u:",len(gamma_u[user])
                print "len gamma i:",len(gamma_i[item])
                print "type gamma u:",type(gamma_u[user])
                print "type gamma i:",type(gamma_i[item])
            obj += np.power((alpha + beta_u[user] + beta_i[item] + np.dot(gamma_u[user],gamma_i[item]) 
                             - userItemRating[user][item]),2)
            count += 1
#     print "obj type:",type(obj)
    # regularization terms
    for user in beta_u:
        obj += lam * math.pow((beta_u[user]),2)
        obj += lam * sum(np.power((gamma_u[user]),2))
#         obj += lam * np.dot(gamma_u[user],gamma_u[user])
    
    for item in beta_i:
        obj += lam * math.pow((beta_i[item]),2)
#         obj += lam * sum(np.power((gamma_i[item]),2))
        obj += lam * np.dot(gamma_i[item],gamma_i[item])
#     print "obj:" , obj
#     print "count;",count 
    return np.float(obj)/count
#     return float(obj)

def grad_alpha(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    grad = 0.0
    count = 0.0
    for user in userItemRating:
        for item in userItemRating[user]:
            grad += 2 * (alpha + beta_u[user] + beta_i[item] + np.dot(gamma_i[item],gamma_u[user]) 
                         - userItemRating[user][item])
            count += 1.0
    return float(grad)/float(count)

def grad_beta_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    grad = {}
    for user in userItemRating:
        grad[user] = 0
        count = 0.0
        for item in userItemRating[user]:
            grad[user] += 2 * (alpha + beta_u[user] + beta_i[item] + np.dot(gamma_i[item],gamma_u[user]) 
                         - userItemRating[user][item])
            count += 1.0
        grad[user] += 2 * lam * beta_u[user]
        grad[user] /= float(count)
        
    return grad

def grad_beta_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    grad = {}
    for item in itemUserRating:
        grad[item] = 0
        count = 0.0
        for user in itemUserRating[item]:
            grad[item] += 2 * (alpha + beta_u[user] + beta_i[item] + np.dot(gamma_i[item],gamma_u[user]) 
                         - itemUserRating[item][user])
            count += 1.0
        grad[item] += 2 * lam * beta_i[item]
        grad[item] /= float(count)
    return grad

def grad_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    grad = {}
    for user in userItemRating:
        grad[user] = np.array([0.0]*K)
        count = 0.0
        for item in userItemRating[user]:
            grad[user] = grad[user] + 2 * (alpha + beta_u[user] + beta_i[item] + np.dot(gamma_i[item],gamma_u[user]) 
                                           - userItemRating[user][item]) * gamma_i[item]
            count += 1.0
        grad[user] = grad[user] + 2 * lam * gamma_u[user]
        grad[user] /= float(count)
    return grad


def grad_gamma_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    grad = {}
#     for item in gamma_i:
#         grad[item] = np.array([0]*K)
#     for user in userItemRating:
#         for item in userItemRating[user]:
#             grad[item] += 2 * (alpha + beta_u[user] + beta_i[item] + np.dot(gamma_i[item],gamma_u[user]) 
#                          - userItemRating[user][item]) * gamma_u[user]
#     for item in gamma_i:
#         grad[item] += 2 * lam * gamma_i[item]
    for item in itemUserRating:
        grad[item] = np.array([0.0]*K)
        count = 0.0
        for user in itemUserRating[item]:
            grad[item] += 2 * (alpha + beta_u[user] + beta_i[item] + np.dot(gamma_i[item],gamma_u[user]) 
                         - userItemRating[user][item]) * gamma_u[user]
            count += 1.0
        grad[item] += 2 * lam * gamma_i[item]
        grad[item] /= float(count)
    return grad
    

In [4]:
def update_alpha(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i):
    grad = 0.0
    user_item_count = 0.0
    for user in userItemRating:
        for item in userItemRating[user]:
            grad += ( userItemRating[user][item] - (beta_u[user] + beta_i[item] + np.dot(gamma_i[item],gamma_u[user])))
            user_item_count += 1
    return float(grad)/float(user_item_count)

def update_beta_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i):
    grad = {}
    for user in userItemRating:
        grad[user] = 0.0
        item_count = 0.0
        for item in userItemRating[user]:
            grad[user] += (userItemRating[user][item] - (alpha + beta_i[item] + np.dot(gamma_i[item],gamma_u[user])))
            item_count += 1
        grad[user] = float(grad[user]) / float(lam + item_count)
    return grad

def update_beta_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i):
    grad = {}
    for item in itemUserRating:
        grad[item] = 0
        count = 0.0
        for user in itemUserRating[item]:
            grad[item] +=  (itemUserRating[item][user] - (alpha + beta_u[user] + np.dot(gamma_i[item],gamma_u[user])))
            count += 1.0
        grad[item] = float(grad[item]) / float(lam + count)
    return grad

def update_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    grad = {}
    for user in userItemRating:
        grad[user] = np.array([0.0]*K)
        first_term = 0.0
        second_term = 0.0
        for item in userItemRating[user]:
            first_term += (gamma_i[item].reshape([K,1]) * gamma_i[item].reshape([1,K]) + lam * np.identity(K)) 
            second_term +=  ( (userItemRating[user][item] 
                              - (alpha + beta_i[item] + beta_u[user]))* gamma_i[item].reshape([K,1]) )
        result = np.dot(np.linalg.inv(first_term), second_term).flatten()
#         print "first:",np.linalg.inv(first_term).shape
#         print "second:",second_term.shape
#         print "result:",result.shape
        grad[user] = result
    return grad

def update_gamma_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K):
    grad = {}
    for item in itemUserRating:
#         grad[item] = np.array([0.0]*K)
        grad[item] = np.array([0.0]*K)
        first_term = 0.0
        second_term = 0.0
        for user in itemUserRating[item]:
            first_term += gamma_u[user].reshape([K,1]) * gamma_u[user].reshape([1,K]) + lam * np.identity(K)
            second_term += (itemUserRating[item][user] - (alpha + beta_i[item] + beta_u[user]))* gamma_u[user].reshape([K,1]) 
        result = np.dot(np.linalg.inv(first_term), second_term).flatten()
#         print result.shape
        grad[item] = result
    return grad

In [5]:
def update_with_gamma_u(itemUserRating,userItemRating,lam
                       ,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss):
    iter = 0
    while (iter < 1000):
        iter += 1
        obj = f(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        loss.append(obj)
        alpha = update_alpha(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i)
        beta_u = update_beta_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i)
        beta_i = update_beta_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i)
        gamma_u = update_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
#         gradient_gamma_i = grad_gamma_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)

        if iter % 1 == 0:
            print "update_with_gamma_u iter :", iter , " loss:",obj
        if len(loss) > 2 and abs(loss[len(loss)-1] - loss[len(loss)-2]) < ACCURACY:
            print "update_with_gamma_u converged iter :", iter , " loss:",obj
            break
    return [alpha,beta_u,beta_i,gamma_u,gamma_i,loss]

def update_with_gamma_i(itemUserRating,userItemRating,lam,
                       alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss):
    iter = 0
    while (iter < 1000):
        iter += 1
        obj = f(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        loss.append(obj)
        alpha = update_alpha(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i)
        beta_u = update_beta_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i)
        beta_i = update_beta_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i)
#         gamma_u = update_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        gamma_i = update_gamma_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)

        if iter % 1 == 0:
            print "update_with_gamma_i iter :", iter , " loss:",obj
        if len(loss) > 2 and abs(loss[len(loss)-1] - loss[len(loss)-2]) < ACCURACY:
            print "update_with_gamma_i converged iter :", iter , " loss:",obj
            break    
    return [alpha,beta_u,beta_i,gamma_u,gamma_i,loss]
    
def train_iterative(itemUserRating,userItemRating,lam,
                       alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,superloss):
    iter = 0
    while (iter < 1000):
        iter += 1
        loss = []
        alpha,beta_u,beta_i,gamma_u,gamma_i,loss = update_with_gamma_u(itemUserRating,userItemRating,lam
                       ,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss)
        superloss.append(loss[len(loss) - 1])
        loss = []
        alpha,beta_u,beta_i,gamma_u,gamma_i,loss = update_with_gamma_i(itemUserRating,userItemRating,lam,
                       alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss)
        superloss.append(loss[len(loss) - 1])
        if len(superloss) > 2 and abs(superloss[len(superloss)-1] - superloss[len(superloss)-2]) < ACCURACY:
            print "training converged at super iter :", iter , " superloss:",superloss[len(superloss)-1]
            break
    return [alpha,beta_u,beta_i,gamma_u,gamma_i,loss]
    

In [6]:
# f(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
# gradient_alpha = grad_alpha(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
# gradient_beta_u = grad_beta_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,user)
# gradient_beta_i = grad_beta_i(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,user)
# gradient_gamma_u = grad_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,user)
# gradient_gamma_i = grad_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,user)

def train_with_gamma_u(itemUserRating,userItemRating,lam
                       ,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr):
    iter = 0
    while (iter < 1000):
        iter += 1
        obj = f(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        loss.append(obj)
        gradient_alpha = grad_alpha(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        gradient_beta_u = grad_beta_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        gradient_beta_i = grad_beta_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        gradient_gamma_u = grad_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
#         gradient_gamma_i = grad_gamma_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)

        if iter % 1 == 0:
            print "train_with_gamma_u iter :", iter , " loss:",obj
        if len(loss) > 2 and abs(loss[len(loss)-1] - loss[len(loss)-2]) < ACCURACY:
            print "train_with_gamma_u converged iter :", iter , " loss:",obj
            break
        
        alpha -= lr * gradient_alpha
        for user in gamma_u:
            beta_u[user] -= lr * gradient_beta_u[user]
            gamma_u[user] -= lr * gradient_gamma_u[user]

        for item in gamma_i:
            beta_i[item] -= lr * gradient_beta_i[item]
#             gamma_i[item] -= lr * gradient_gamma_i[item]
    return [alpha,beta_u,beta_i,gamma_u,gamma_i,loss]

def train_with_gamma_i(itemUserRating,userItemRating,lam,
                       alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr):
    iter = 0
    while (iter < 1000):
        iter += 1
        obj = f(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        loss.append(obj)
        gradient_alpha = grad_alpha(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        gradient_beta_u = grad_beta_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        gradient_beta_i = grad_beta_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
#         gradient_gamma_u = grad_gamma_u(userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)
        gradient_gamma_i = grad_gamma_i(itemUserRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K)

        if iter % 1 == 0:
            print "train_with_gamma_i iter :", iter , " loss:",obj
        if len(loss) > 2 and abs(loss[len(loss)-1] - loss[len(loss)-2]) < ACCURACY:
            print "train_with_gamma_i converged iter :", iter , " loss:",obj
            break
        
        
            
        alpha -= lr * gradient_alpha
        for user in gamma_u:
            beta_u[user] -= lr * gradient_beta_u[user]
#             gamma_u[user] -= lr * gradient_gamma_u[user]

        for item in gamma_i:
            beta_i[item] -= lr * gradient_beta_i[item]
            gamma_i[item] -= lr * gradient_gamma_i[item]
    return [alpha,beta_u,beta_i,gamma_u,gamma_i,loss]

def train(itemUserRating,userItemRating,lam,
                       alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr,superloss):
    iter = 0
    while (iter < 1000):
        iter += 1
        [alpha,beta_u,beta_i,gamma_u,gamma_i,loss] = train_with_gamma_u(itemUserRating,userItemRating,lam
                       ,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr)
        superloss.append(loss[len(loss) - 1])
        
        [alpha,beta_u,beta_i,gamma_u,gamma_i,loss] = train_with_gamma_i(itemUserRating,userItemRating,lam,
                       alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr)
        superloss.append(loss[len(loss) - 1])
        
        if len(superloss) > 2 and abs(superloss[len(superloss)-1] - superloss[len(superloss)-2]) < ACCURACY:
            print "training converged at super iter :", iter , " superloss:",superloss[len(superloss)-1]
            return superloss[len(superloss)-1]

def write_Lambda_K_Variations(list):
    predictions = open("lambda_k_variations.txt", 'w')
    for l in list:
        predictions.write(l)
    predictions.close()
    
def write_predictions_Rating(alpha,beta_u,beta_i,gamma_u,gamma_i):
    predictions = open("predictions_Rating.txt", 'w')
    for l in open("pairs_Rating.txt"):
        if l.startswith("userID"):
            predictions.write(l) #header
            continue
        u,i = l.strip().split('-')
        predictions.write(u + '-' + i + ',' + str(predict_rating(u,i,4.218,alpha,beta_u,beta_i,gamma_u,gamma_i)) + '\n')
    predictions.close()
    
def predict_rating(user,item,avg,alpha,beta_u,beta_i,gamma_u,gamma_i):
    if user in beta_u and item in beta_i:
        pred = alpha + beta_u[user]+beta_i[item] + np.dot(gamma_u[user],gamma_i[item])
#         print "pred:",pred
        return pred
    else:
        return avg
    
def getMSE(userItemRating,avg,alpha,beta_u,beta_i,gamma_u,gamma_i):
    se = 0.0
    count = 0
    for user in userItemRating:
        for item in userItemRating[user]:
            pred = predict_rating(user,item,avg,alpha,beta_u,beta_i,gamma_u,gamma_i)
            se += math.pow((pred - userItemRating[user][item]),2)
            count += 1
    return float(se)/float(count)


In [7]:
lam = 1
lambdas = [5]
# lambdas = [10]
lrs =      [0.05 for i in range(len(lambdas))]
Ks = [5]
# Ks = [5]
alpha = random.random()
K = 5        #dimensions of latent factor model
beta_u  = {} #scalar value for each user
beta_i  = {} #scalar value for each item
gamma_u = {} #K dim vector for each user
gamma_i = {} #K dim vector for each item
# lr = 0.000005
lr = 0.1
loss = []
superloss = []
ACCURACY = 0.001

#initialize model parameters
def initializeModelParams(itemUserRating,userItemRating,beta_i,beta_u,gamma_i,gamma_u,K):
    for user in userItemRating:
        beta_u[user] = random.random()
#         gamma_u[user] = np.matrix(np.array([random.random() for i in range(K)]).reshape([K,1]))
        gamma_u[user] = np.array([random.random() for i in range(K)])
        for item in userItemRating[user]:
            if item not in beta_i:
                beta_i[item]= random.random()
            if item not in gamma_i:
                gamma_i[item]=np.array([random.random() for i in range(K)])
    alpha = pickle.load( open( "alpha.p", "rb" ) )
    beta_u = pickle.load( open( "beta_u.p", "rb" ) )
    beta_i = pickle.load( open( "beta_i.p", "rb" ) )
                

In [264]:
# out = []
# out.append("lam\tK\ttrain_loss\ttest_loss\n")
# avg = 4.19
# for i in range(len(lambdas)):
#     lam = lambdas[i]
#     lr = lrs[i]
#     print "lr:",lr
#     for K in Ks:
#         beta_u  = {} #scalar value for each user
#         beta_i  = {} #scalar value for each item
#         gamma_u = {} #K dim vector for each user
#         gamma_i = {} #K dim vector for each item
#         alpha = random.random()
#         loss = []
#         superloss = []
#         initializeModelParams(itemUserRating_100k,userItemRating_100k,beta_i,beta_u,gamma_i,gamma_u,K)
#         train(itemUserRating_100k,userItemRating_100k,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr,superloss)
#         train_loss = getMSE(userItemRating_100k,avg,alpha,beta_u,beta_i,gamma_u,gamma_i)
#         test_loss = getMSE(userItemRating_900k,avg,alpha,beta_u,beta_i,gamma_u,gamma_i)
#         out.append(str(lam) + "\t" + str(K) + "\t" + str(train_loss) + "\t" + str(test_loss) + "\n")
#         print "TRAINING complete WITH K:", K , " LAMBDA:", lam
#         write_Lambda_K_Variations(out)



In [8]:
out = []
out.append("lam\tK\ttrain_loss\ttest_loss\n")
avg = 4.19
beta_u  = {} #scalar value for each user
beta_i  = {} #scalar value for each item
gamma_u = {} #K dim vector for each user
gamma_i = {} #K dim vector for each item
alpha = random.random()
for i in range(len(lambdas)):
    lam = lambdas[i]
    for K in Ks:
        beta_u  = {} #scalar value for each user
        beta_i  = {} #scalar value for each item
        gamma_u = {} #K dim vector for each user
        gamma_i = {} #K dim vector for each item
        alpha = pickle.load( open( "alpha.p", "rb" ) )
        loss = []
        superloss = []
#         initializeModelParams(itemUserRating_100k,userItemRating_100k,beta_i,beta_u,gamma_i,gamma_u,K)
#         alpha,beta_u,beta_i,gamma_u,gamma_i,loss = train_iterative(itemUserRating_100k,userItemRating_100k,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,superloss)
        initializeModelParams(itemUserRating,userItemRating,beta_i,beta_u,gamma_i,gamma_u,K)
        alpha,beta_u,beta_i,gamma_u,gamma_i,loss = train_iterative(itemUserRating,userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,superloss)        
        train_loss = getMSE(userItemRating_100k,avg,alpha,beta_u,beta_i,gamma_u,gamma_i)
        test_loss = getMSE(userItemRating_900k,avg,alpha,beta_u,beta_i,gamma_u,gamma_i)
        out.append(str(lam) + "\t" + str(K) + "\t" + str(train_loss) + "\t" + str(test_loss) + "\n")
        print "TRAINING complete WITH K:", K , " LAMBDA:", lam
        write_Lambda_K_Variations(out)

update_with_gamma_u iter : 1  loss: 7.05444203985
update_with_gamma_u iter : 2  loss: 2.26510469825
update_with_gamma_u iter : 3  loss: 1.09981143879
update_with_gamma_u iter : 4  loss: 1.01939677353
update_with_gamma_u iter : 5  loss: 1.00390069176
update_with_gamma_u iter : 6  loss: 0.9956228165
update_with_gamma_u iter : 7  loss: 0.989260370734
update_with_gamma_u iter : 8  loss: 0.983951109904
update_with_gamma_u iter : 9  loss: 0.979459229031
update_with_gamma_u iter : 10  loss: 0.975666105956
update_with_gamma_u iter : 11  loss: 0.972480149579
update_with_gamma_u iter : 12  loss: 0.969819927161
update_with_gamma_u iter : 13  loss: 0.967611397565
update_with_gamma_u iter : 14  loss: 0.96578773217
update_with_gamma_u iter : 15  loss: 0.964289397903
update_with_gamma_u iter : 16  loss: 0.963064060554
update_with_gamma_u iter : 17  loss: 0.962066272161
update_with_gamma_u converged iter : 17  loss: 0.962066272161
update_with_gamma_i iter : 1  loss: 0.961256993496
update_with_gamma_i 

In [221]:
loss = []
superloss = []
beta_u  = {} #scalar value for each user
beta_i  = {} #scalar value for each item
gamma_u = {} #K dim vector for each user
gamma_i = {} #K dim vector for each item
K = 3
lr = 0.005
lam = 10
alpha = random.random()
# initializeModelParams(itemUserRating,userItemRating,beta_i,beta_u,gamma_i,gamma_u,K)
# train(itemUserRating,userItemRating,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr,superloss)
initializeModelParams(itemUserRating_100k,userItemRating_100k,beta_i,beta_u,gamma_i,gamma_u,K)
train(itemUserRating_100k,userItemRating_100k,lam,alpha,beta_u,beta_i,gamma_u,gamma_i,K,loss,lr,superloss)
train_loss = getMSE(userItemRating_100k,avg,alpha,beta_u,beta_i,gamma_u,gamma_i)
test_loss = getMSE(userItemRating_900k,avg,alpha,beta_u,beta_i,gamma_u,gamma_i)

In [10]:
write_predictions_Rating(alpha,beta_u,beta_i,gamma_u,gamma_i)

In [51]:
# import numpy as np
# a = np.matrix(np.ones(4))
# a=  np.random.random([3,3])
# print a 
# # b = np.invert(a)

# c = np.linalg.inv(np.identity(5))

# c * np.identity(5)

# print userItemRating[user]
user = 'U732551804'
item = 'I238406892'
print np.power((alpha + beta_u[user] + beta_i[item] + np.dot(gamma_u[user],gamma_i[item]) 
                             - userItemRating[user][item]),2)

7.12633356236
